In [1]:
import pandas as pd
import numpy as np

from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
import evaluate
import torch
from datasets import Dataset
import random


df = pd.read_json('synthetic_10000.jsonl', lines=True) 

ASPECTS = [
    "Cost of Living", "Healthcare", "Education", "Personal Security",
    "Employment", "Transportation", "Government", "Environment",
    "Social Equality", "Taxation"
]

labels_expanded = df["labels"].apply(pd.Series)
df = pd.concat([df.drop(columns=["labels"]), labels_expanded], axis=1)

def transform_to_absa_format(original_df):
    transformed_rows = []
    
    for index, row in original_df.iterrows():
        tweet = row['tweet_text']
        
        active_aspects = []
        inactive_aspects = []
        
        for aspect in ASPECTS:
            if row[aspect] != 0:
                active_aspects.append(aspect)
            else:
                inactive_aspects.append(aspect)
        
        for aspect in active_aspects:
            sentiment = row[aspect]
            label = 0 if sentiment == -1 else 2
            
            transformed_rows.append({
                'text': tweet, 
                'aspect': aspect, 
                'label': label
            })
            
        num_negatives_to_sample = len(active_aspects) 
        if num_negatives_to_sample > 0 and len(inactive_aspects) > 0:
            sampled_inactives = random.sample(inactive_aspects, k=min(num_negatives_to_sample, len(inactive_aspects)))
            for aspect in sampled_inactives:
                transformed_rows.append({
                    'text': tweet, 
                    'aspect': aspect, 
                    'label': 1 
                })
            
    return pd.DataFrame(transformed_rows)

absa_df = transform_to_absa_format(df)
print("ABSA rows:", len(absa_df))
print("Label distribution:\n", absa_df["label"].value_counts())

hf_dataset = Dataset.from_pandas(absa_df)
dataset_dict = hf_dataset.train_test_split(test_size=0.2, seed=42)
print("Train size:", len(dataset_dict["train"]))
print("Test size:", len(dataset_dict["test"]))

print(f"Original rows: {len(df)}")
print(f"Training samples (aspect-level): {len(dataset_dict['train'])}")

c:\Users\reidu\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ABSA rows: 30004
Label distribution:
 label
1    15002
2     7516
0     7486
Name: count, dtype: int64
Train size: 24003
Test size: 6001
Original rows: 10000
Training samples (aspect-level): 24003


In [ ]:
model_id = "yangheng/deberta-v3-base-absa-v1.1" 
tokenizer = AutoTokenizer.from_pretrained(model_id)

def tokenize_function(examples):
    return tokenizer(
        examples['text'],     
        examples['aspect'],   
        padding="max_length", 
        truncation=True, 
        max_length=128
    )

tokenized_datasets = dataset_dict.map(tokenize_function, batched=True)

model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=3)

metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    pred = np.argmax(logits, axis=-1)
    
    acc = metric.compute(predictions=pred, references=labels)["accuracy"]
    f1_macro = f1_metric.compute(
        predictions=pred,
        references=labels,
        average="macro"
    )["f1"]

    return {
        "accuracy": acc,
        "f1_macro": f1_macro,
    }

training_args = TrainingArguments(
    output_dir="./absa_finetuned_model",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,      
    weight_decay=0.1,       
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,      
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    logging_steps=100,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=1)],
)

print("Starting training...")
trainer.train()

trainer.save_model("./final_absa_model")
tokenizer.save_pretrained("./final_absa_model")
print("Model saved successfully.")

c:\Users\reidu\AppData\Local\Programs\Python\Python313\Lib\site-packages\transformers\convert_slow_tokenizer.py:566: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Map: 100%|██████████| 6001/6001 [00:00<00:00, 8949.42 examples/s]


Starting training...


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro
1,0.454300,0.423814,0.830362,0.798556
2,0.366200,0.459302,0.824529,0.788997


Model saved successfully.


In [ ]:
from transformers import pipeline

absa_pipeline = pipeline(
    "text-classification", 
    model="./final_absa_model", 
    tokenizer="./final_absa_model",
    device=0 if torch.cuda.is_available() else -1,
    top_k=None
)

def analyze_tweet(text: str, aspect_list):
    print(f"\nTweet: {text!r}")
    label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}

    model.eval()
    device = model.device

    for asp in aspect_list:
        inputs = tokenizer(text, asp, return_tensors="pt").to(device)
        with torch.no_grad():
            logits = model(**inputs).logits
            probs = torch.softmax(logits, dim=-1)[0]
            cls = torch.argmax(probs).item()
            max_p = probs[cls].item()

        if cls != 1 and max_p > 0.6:
            print(f"  Aspect: {asp:<20} -> {label_map[cls]} ({max_p:.2f})")

if __name__ == "__main__":
    sample = "The taxes are killing us, but at least the streets are safe at night."
    analyze_tweet(sample, ASPECTS)

The tokenizer you are loading from './final_absa_model' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.
Device set to use cuda:0



Tweet: 'The taxes are killing us, but at least the streets are safe at night.'
  Aspect: Cost of Living       -> Negative (0.60)
  Aspect: Personal Security    -> Positive (0.91)
  Aspect: Taxation             -> Negative (0.81)
